In [4]:
from IPython.display import display, HTML
from string import Template
import json
from typing import List,Dict
import jsons

In [5]:
#data = Data([DataSet("test_Data",list([DataPoint(x=1,y=1),DataPoint(x=2,y=2),DataPoint(x=3,y=3)]), "000000")])

In [8]:
%load_ext blackcellmagic

In [11]:
#%%black
#%%writefile crawto/Charts/Charts.py
import uuid

default_colorscheme = [
    "#4E79A7",
    "#F28E2B",
    "#E15759",
    "#76B7B2",
    "#59A14F",
    "#EDC948",
    "#B07AA1",
    "#FF9DA7",
    "#9C755F",
    "#BAB0AC",
]


class Chart:
    def __init__(
        self,
        data: Dict[str, List] = None,
        title: Dict[str, str] = None,
        colorscheme: List[str] = None,
    ):
        self.id = uuid.uuid1()
        self.data = data if data is not None else {"datasets": []}
        self.colorscheme = "tableauClassic10"
        self.title = title if title is not None else {"display": "false", "text": ""}
        self.colorscheme = (
            colorscheme if colorscheme is not None else default_colorscheme
        )
        self.xAxes = [
            {"display": "true", "scaleLabel": {"display": "false", "labelString": ""}}
        ]
        self.yAxes = [
            {
                "display": "true",
                "scaleLabel": {"display": "false", "labelString": ""},
                "type": "linear",
            }
        ]

    def add_colors(self):
        l = len(self.colorscheme)
        for i, j in enumerate(self.data["datasets"]):
            j["backgroundColor"] = self.colorscheme[i % l]
            j["borderColor"] = self.colorscheme[i % l]
        return self.data

    @property
    def html(self):
        html = Template(
            """
        <canvas id= "$id" ></canvas>
        <script>
        requirejs(['https:\\cdnjs.cloudflare.com/ajax/libs/Chart.js/2.8.0/Chart.js'], function(Chart){
            new Chart(document.getElementById("$id"), {
                    type: "$type",
                    data: $data,
                    options: {
                        "responsive": true,
                        "title": $title,
                        "scales" : { xAxes : $xAxes,
                                     yAxes : $yAxes
                                     },
                        tooltips: {
                            callbacks: {
                                label: function(tooltipItem, data) {
                                    var label = data.datasets[tooltipItem.datasetIndex].label;
                                    var word = data.datasets[tooltipItem.datasetIndex].data[tooltipItem.index].Value
                                    return word == undefined? label : label + " : " + word
                               }
                            }
                        }           
                    }
                });
            }); 
        </script>
        """
        )
        self.add_colors()
        html = html.substitute(
            {
                "data": jsons.dumps(self.data),
                "id": self.id,
                "title": jsons.dumps(self.title),
                "xAxes": jsons.dumps(self.xAxes),
                "yAxes": jsons.dumps(self.yAxes),
                "type": self.type,
            }
        )
        return html

    def edit_title(self, text: str):
        new_dict = {"display": "true", "text": text}
        self.title.update(new_dict)

    def edit_xAxes(self, text: str = "", axisIndex: int = 0):
        self.xAxes[axisIndex]["scaleLabel"]["display"] = "true"
        self.xAxes[axisIndex]["scaleLabel"]["labelString"] = text

    def edit_yAxes(self, text: str = "", type: str = "linear", axisIndex: int = 0):
        self.yAxes[axisIndex]["scaleLabel"]["display"] = "true"
        self.yAxes[axisIndex]["scaleLabel"]["labelString"] = text
        self.yAxes[axisIndex]["type"] = type

    def __repr__(self):
        return self.html


class ScatterChart(Chart):
    @property
    def type(self):
        return "scatter"

    def add_DataSet(self, label: str, x, y, unique_identifier=None):
        x, y, = list(x), list(y)
        u = list(unique_identifier) if unique_identifier is not None else None
        if len(x) != len(y):
            raise Exception("x and y columns are not equal in length")

        d = {
            "label": label,
            "data": [{"x": float(x[i]), "y": float(y[i])} for i in range(len(x))],
        }
        if u is not None:
            if len(u) != len(y):
                raise Exception(
                    "unique_identifier and x or y columns are not equal in length"
                )
            else:
                for i, j in enumerate(d["data"]):
                    j["Value"] = u[i]

        self.data["datasets"].append(d)


class BarChart(Chart):
    @property
    def type(self):
        return "bar"

    def add_DataSet(self, label: str, x, y):
        if "labels" not in self.data.keys():
            self.data["labels"] = list([str(i) for i in x])
        elif list([str(i) for i in x]) != self.data["labels"]:
            raise Exception(f"Already defined the labels for this chart")

        self.data["datasets"].append({"label": label, "data": list(y)})


class LineChart(Chart):
    @property
    def type(self):
        return "line"

    def add_DataSet(self, label: str, x, y, fill: str = "false"):
        if "labels" not in self.data.keys():
            self.data["labels"] = list([str(i) for i in x])
        elif list([str(i) for i in x]) != self.data["labels"]:
            raise Exception(f"Already defined the labels for this chart")

        self.data["datasets"].append({"label": label, "data": list(y), "fill": fill})

In [12]:
x=[-1,0,1]
y = [1,-1,1]

l = LineChart()
l.add_DataSet("t",x,y)
l.add_DataSet("t1" ,x,[2,2,2])
l.edit_title("LineChart")
l.edit_xAxes("x Axis")
l.edit_yAxes("y Axis","linear")

c = ScatterChart()
c.add_DataSet("test",x,y)
c.add_DataSet("t1" ,x,[2,2,2],["a","b","c"])
c.edit_title("ScatterChart")

b = BarChart()
b.add_DataSet("t",x,y)
b.add_DataSet("t1" ,x,[2,2,2])
b.edit_title("BarChart")

In [13]:
from typing import Union

Chart = Union[ScatterChart]
class Plot:    
    def __init__(self,):
        self.head = """
        <head>
                <script type="application/javascript" 
                src="https:\\cdnjs.cloudflare.com/ajax/libs/require.js/x.y.z/require.js"></script>
                <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/semantic-ui@2.4.2/dist/semantic.min.css">
                <script src="https://code.jquery.com/jquery-3.1.1.min.js"
                  integrity="sha256-hVVnYaiADRTO2PzUGmuLJr8BLUSjGIZsDYGmIJLv2b8="
                  crossorigin="anonymous"></script>
                <script src="https://cdn.jsdelivr.net/npm/semantic-ui@2.4.2/dist/semantic.min.js"></script>
        </head>
        """
        self.body = """
        <body>
        <div class= "ui grid">
        """
    
    def add_column(self,chart:Chart,width:str):
        html = f"<div class = '{width} wide column'>\n"
        html +=chart.html + "\n"
        html +="\n</div>\n"
        self.body+=html
            
    
    def display(self):
        d= self.head+ self.body + "\n</div>\n</body>"
        return HTML(d)
        

In [14]:
p = Plot()

In [15]:
p.add_column(c,"eight")
p.add_column(b,"eight")
p.add_column(l,"eight")

In [16]:
p.display()